## Dask demo notebook

This notebook shows a demonstration of how to use dask and CMAC2.0 on a JupyterHub node.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import dask.bag as db
import pyart
import importlib
import netCDF4
import os
import subprocess
import sys
import imageio
import numpy as np

from glob import glob
from cmac import cmac, quicklooks, get_sounding_times, get_sounding_file_name, config
from IPython.display import Image, display
from dask_jobqueue import SLURMCluster, PBSCluster
from datetime import datetime
from distributed import Client
%matplotlib inline


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



This subroutine, will do both the processing and plotting for one radar file. A more sophisicated version of this is contained within the cmac_dask script.

In [2]:
def run_cmac_and_plotting(radar_file_path, radar_config,
                          sonde_file, clutter_file_path, 
                          out_path, img_directory, sweep=0, dd_lobes=False):
    """ For dask we need the radar plotting routines all in one subroutine. """
    try:
        radar = pyart.io.read(radar_file_path)
    except TypeError:
        print(radar_file_path + ' has encountered TypeError!')
        return

    
    radar_start_date = netCDF4.num2date(radar.time['data'][0],
                                        radar.time['units'])
    year_str = "%04d" % radar_start_date.year
    month_str = "%02d" % radar_start_date.month
    day_str = "%02d" % radar_start_date.day
    hour_str = "%02d" % radar_start_date.hour
    minute_str = "%02d" % radar_start_date.minute
    second_str = "%02d" % radar_start_date.second
    save_name = cmac_config['save_name']
    file_name = (out_path + '/' + save_name + '.'
                 + year_str + month_str + day_str + '.' + hour_str
                 + minute_str + second_str + '.nc')

    # Load clutter files.
    clutter = pyart.io.read(clutter_file_path)
    clutter_field_dict = clutter.fields['ground_clutter']
    radar.add_field(
        'clutter', clutter_field_dict, replace_existing=True)
    del clutter

    # Retrieve closest sonde in time to the time of the radar file.
    sonde = netCDF4.Dataset(sonde_file)

    # Running the cmac code to produce a cmac_radar object.
    cmac_radar = cmac(radar, sonde, 'cacti_csapr2_ppi')
    # Free up some memory.
    del radar
    sonde.close()

    # Produce the cmac_radar file from the cmac_radar object.
    pyart.io.write_cfradial(file_name, cmac_radar)
    print('## A CMAC radar object has been created at ' + file_name)

    if not os.path.exists(img_directory):
        os.makedirs(img_directory)
        subprocess.call('chmod -R g+rw ' + img_directory, shell=True)

    # Producing all the cmac_radar quicklooks.
    quicklooks(cmac_radar, 'cacti_csapr2_ppi',
               dd_lobes=False, image_directory=img_directory)

    # Delete the cmac_radar object and move on to the next radar file.
    del cmac_radar
    return

We have to start our dask cluster. Normally, the script to do this on stratus is in qsub_xsapr. The dask-scheduler has to be running on one compute node and the dask-workers on the other computer nodes. However, since we are only on one node, we can just use multiprocessing to do our work for us.

CMAC2.0 has different dictionaries that specify various configurations for the 3 XSAPRs in SGP. Here we will load i5 since the demo data is from XSAPR i5.

In [4]:
meta_config = config.get_metadata('cacti_csapr2_ppi')
cmac_config = config.get_cmac_values('cacti_csapr2_ppi')
field_config = config.get_field_names('cacti_csapr2_ppi')


We can then use a dask bag to map the radar list into distributed memory and then execute the processing code on each file using .map().compute() on the bag

In [5]:
# # on bebop-------------------------------------------
# soundings_directory = '/home/icrisologo/Data/CACTI/sonde/'
# radar_directory = '/lcrc/group/earthscience/radar/CACTI/PPI/'
# clutter_directory = '/lcrc/group/earthscience/icrisologo/cmac/clutter/'

# img_directory = '/lcrc/group/earthscience/icrisologo/csapr_imgs/'
# out_path = '/lcrc/group/earthscience/icrisologo/csapr_files/'
# log_path = '/lcrc/group/earthscience/icrisologo/csapr_log/'

# on stratus-------------------------------------------
radar_directory = '/lustre/or-hydra/cades-arm/proj-shared/corcsapr2cfrppiM1.a1/201901/'
soundings_directory = '/lustre/or-hydra/cades-arm/proj-shared/corsondewnpnM1.b1/'
clutter_directory = '/lustre/or-hydra/cades-arm/proj-shared/irene/cmac/clutter/'

img_directory = '/lustre/or-hydra/cades-arm/proj-shared/irene/cmac/csapr_imgs/'
out_path = '/lustre/or-hydra/cades-arm/proj-shared/irene/cmac/csapr_files/'
log_path = '/lustre/or-hydra/cades-arm/proj-shared/irene/cmac/csapr_log/'


radar_files = glob(radar_directory + '*')
sounding_files = glob(soundings_directory + '*')
clutter_files = glob(clutter_directory + '*')

In [6]:
radar_files.sort()
sounding_files.sort()
clutter_files.sort()

In [7]:
def parse_sonde_times(file_list):
    time_list = []
    for name in file_list:
        time_list.append(datetime.strptime(name.split('/')[-1], 'corsondewnpnM1.b1.%Y%m%d.%H%M%S.cdf'))
    return np.array(time_list)
        
def parse_clutter_times(file_list):
    time_list = []
    for name in file_list:
        time_list.append(datetime.strptime(name.split('/')[-1], 'clutter_corcsapr2cfrppiM1.a1.%Y%m%d.%H%M%S.nc'))
    return np.array(time_list)

sonde_times = parse_sonde_times(sounding_files)
clutter_times = parse_clutter_times(clutter_files)

In [8]:
clutter_times.sort()
sonde_times.sort()

In [9]:
rad_file = radar_files[0]
radar = pyart.io.read(rad_file)
rad_time = datetime.strptime(radar.time["units"][0:33], "seconds since %Y-%m-%d %H:%M:%S")

In [10]:
sonde_index = np.argmin(np.abs(sonde_times - rad_time))
sounding_file = sounding_files[sonde_index]

In [11]:
clutter_index = np.argmin(np.abs(clutter_times - rad_time))
clutter_file = clutter_files[clutter_index]

In [12]:
print(sounding_file)
print(clutter_file)

/lustre/or-hydra/cades-arm/proj-shared/corsondewnpnM1.b1/corsondewnpnM1.b1.20190121.160000.cdf
/lustre/or-hydra/cades-arm/proj-shared/irene/cmac/clutter/clutter_corcsapr2cfrppiM1.a1.20190121.174504.nc


In [13]:
def run_cmac_from_index(i):
    print(i)
    rad_file = radar_files[i]
    radar = pyart.io.read(rad_file)
    rad_time = datetime.strptime(radar.time["units"][0:33], "seconds since %Y-%m-%d %H:%M:%S")
    del radar
    sonde_index = np.argmin(np.abs(sonde_times - rad_time))
    sounding_file = sounding_files[sonde_index]
    clutter_index = np.argmin(np.abs(clutter_times - rad_time))
    clutter_file = clutter_files[clutter_index]
    
    f= open(os.path.join(log_path,"{0}.txt".format(str(i).zfill(4))),"w+")
    f.write('Radar file: {0}\n'.format(rad_file))
    f.write('Sounding file: {0}\n'.format(sounding_file))
    f.write('Clutter file: {0}\n'.format(clutter_file))
    f.close()
            
    run_cmac_and_plotting(rad_file, 'cacti_csapr2_ppi',
                          sounding_file, clutter_file, 
                          out_path, img_directory)

In [15]:
# not official parameters
cluster1 = PBSCluster(processes=12, cores=12, walltime='3:00:00', queue='batch', memory='128GB')
cluster1.scale(12*12)         # Ask for ten workers
client1 = Client(cluster1)

In [19]:
client1

Client Scheduler: tcp://172.23.82.81:38417 Dashboard: http://172.23.82.81:36013/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [14]:
#run_cmac_from_index(0)

In [20]:
# on bebop--------------------------------------
# my_cluster = SLURMCluster(cores=36, processes=4, walltime='2:00:00', memory='128GB')
# my_cluster.scale(12)

# on stratus
# dont use queue high-mem , use batch
# my_cluster = PBSCluster(cores=36, processes=6, walltime='2:00:00', memory='270GB',
#                        name='dask-worker', interface='ib0', queue='batch', project='arm',
#                        job_extra=['-W group_list=cades-arm'])
my_cluster = PBSCluster(
    name='dask-worker',
    cores=36,
    memory='256GB',
    processes=6,
    local_directory='$localscratch',
    queue='batch',
    project='arm',
    interface='ib0',
    walltime='00:10:00',
    job_extra=['-W group_list=cades-arm'],
    scheduler_port=8888
)
my_cluster.scale(12*12)

In [21]:
client = Client(my_cluster)

In [24]:
client

Client Scheduler: tcp://10.23.216.81:39079 Dashboard: http://10.23.216.81:38942/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


On stratus terminal, check the job queue:
`$ qstat -u irene`

In [22]:
my_cluster

dask-worker('tcp://10.23.216.81:39324', workers=0, threads=0, memory=0 B)

In [24]:
print(client.ncores())

{}


In [25]:
print("Waiting for cluster...")
while client.ncores() == {}:
    continue
print("Executing..")
the_bag = db.from_sequence(np.arange(0, len(radar_files)))

# Wrap your function into a lambda function so that .map can use it!
result = the_bag.map(run_cmac_from_index).compute()

Waiting for cluster...


KeyboardInterrupt: 

In [ ]:
rad_file = radar_files[540]
radar = pyart.io.read(rad_file)

In [ ]:
rad_time = datetime.strptime(radar.time["units"][0:33], "seconds since %Y-%m-%d %H:%M:%S")
sonde_index = np.argmin(np.abs(sonde_times - rad_time))
sounding_file = sounding_files[sonde_index]
clutter_index = np.argmin(np.abs(clutter_times - rad_time))
clutter_file = clutter_files[clutter_index]

In [ ]:
clutter = pyart.io.read(clutter_file)
clutter_field_dict = clutter.fields['ground_clutter']

In [ ]:
clutter_field_dict['data'].shape

In [ ]:
for key in radar.fields.keys():
    print(radar.fields[key]['data'].shape)

# WHY am i getting an invalid shape error? Where is the shape (5402,1110) coming from?

In [ ]:
prof.visualize()

In [ ]:
# im_files = glob('/home/rjackson/xsapr_imgs/reflectivity*.png')
# im_files.sort()
# images = []
# for filename in im_files:
#     images.append(imageio.imread(filename))
# imageio.mimsave('refl_animation.gif', images, duration=0.5)

# with open('refl_animation.gif','rb') as f:
#     display(Image(f.read()))

In [ ]:
im_files = glob('/lcrc/group/earthscience/icrisologo/csapr_imgs/masked_corrected_reflectivity*.png')
im_files.sort()
images = []
for filename in im_files:
    images.append(imageio.imread(filename))
imageio.mimsave('refl_animation.gif', images, duration=0.5)

with open('refl_animation.gif','rb') as f:
    display(Image(f.read()))